## Predicting Service Disruptions

### A telecom company is interested in developing an advanced predictive model topredict service disruptions based on the log files generated by multiple devices.We have to output a csv file that can be handed to the operations team so theycan priorities dispatch of technicians based on fault_severity prediction and itsprobability.

In [1]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [2]:
train=pd.read_csv('train.csv')
event=pd.read_csv('event_type.csv')
log=pd.read_csv('log_feature.csv')
rsc=pd.read_csv('resource_type.csv')
svr=pd.read_csv('severity_type.csv')

In [3]:
train.shape, event.shape, log.shape, svr.shape, rsc.shape

((7381, 3), (31170, 2), (58671, 3), (18552, 2), (21076, 2))

In [4]:
train1= train.merge(svr, how = 'left', left_on = 'id', right_on = 'id' )
train2=train1.merge(rsc, how = 'left', left_on = 'id', right_on = 'id' )
train3=train2.merge(log, how = 'left', left_on = 'id', right_on = 'id' )
train4=train3.merge(event, how = 'left', left_on = 'id', right_on = 'id' )

In [5]:
train4.head()

,id,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 34
1,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 35
2,14121,location 118,1,severity_type 2,resource_type 2,feature 232,19,event_type 34
3,14121,location 118,1,severity_type 2,resource_type 2,feature 232,19,event_type 35
4,9320,location 91,0,severity_type 2,resource_type 2,feature 315,200,event_type 34


In [6]:
train4.shape

(61839, 8)

In [7]:
train4.isnull().sum()

id                0
location          0
fault_severity    0
severity_type     0
resource_type     0
log_feature       0
volume            0
event_type        0
dtype: int64

In [8]:
train4.describe().T

,count,mean,std,min,25%,50%,75%,max
id,61839.0,9104.379469,5387.274195,1.0,4353.0,8981.0,13664.0,18550.0
fault_severity,61839.0,0.549168,0.727701,0.0,0.0,0.0,1.0,2.0
volume,61839.0,8.390336,22.902491,1.0,1.0,2.0,6.0,877.0


In [9]:
train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61839 entries, 0 to 61838
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              61839 non-null  int64 
 1   location        61839 non-null  object
 2   fault_severity  61839 non-null  int64 
 3   severity_type   61839 non-null  object
 4   resource_type   61839 non-null  object
 5   log_feature     61839 non-null  object
 6   volume          61839 non-null  int64 
 7   event_type      61839 non-null  object
dtypes: int64(3), object(5)
memory usage: 4.2+ MB


In [10]:
train4.duplicated(subset='id').sum()
# id sütununda totalde kaç tekrar eden değer olduğunu gösterir

54458

In [11]:
train4.drop_duplicates(subset='id', keep= 'first', inplace=True)
#tekrar eden değerleri satırları atıyoruz

In [12]:
train4.shape

(7381, 8)

In [13]:
cat_col=list(set(train4.columns)-set(train4._get_numeric_data().columns))
cat_col
# we find the object columns

['resource_type', 'log_feature', 'location', 'event_type', 'severity_type']

In [14]:
train4.head()

,id,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 34
4,9320,location 91,0,severity_type 2,resource_type 2,feature 315,200,event_type 34
8,14394,location 152,1,severity_type 2,resource_type 2,feature 221,1,event_type 35
12,8218,location 931,1,severity_type 1,resource_type 8,feature 80,9,event_type 15
18,14804,location 120,0,severity_type 1,resource_type 2,feature 134,1,event_type 34


In [15]:
train4.replace({'location ': '', 'severity_type ': '', 'resource_type ': '', 'feature ': '', 'event_type': ''}, regex=True, inplace=True)
# we dumped string values from object columns

In [16]:
train4.head()

,id,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,14121,118,1,2,2,312,19,34
4,9320,91,0,2,2,315,200,34
8,14394,152,1,2,2,221,1,35
12,8218,931,1,1,8,80,9,15
18,14804,120,0,1,2,134,1,34


In [17]:
def convertObToCat(df,cat_col):
    for i in range(len(cat_col)):
        df[cat_col[i]]=df[cat_col[i]].astype('category')
    return df
# We convert object data into category

In [18]:
train4=convertObToCat(train4, cat_col)
train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7381 entries, 0 to 61836
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   id              7381 non-null   int64   
 1   location        7381 non-null   category
 2   fault_severity  7381 non-null   int64   
 3   severity_type   7381 non-null   category
 4   resource_type   7381 non-null   category
 5   log_feature     7381 non-null   category
 6   volume          7381 non-null   int64   
 7   event_type      7381 non-null   category
dtypes: category(5), int64(3)
memory usage: 332.5 KB


In [19]:
y=train4[['fault_severity']]
y.head()

,fault_severity
0,1
4,0
8,1
12,1
18,0


In [20]:
train4.drop(['fault_severity','id'], inplace=True, axis=1)
train4.head()

,location,severity_type,resource_type,log_feature,volume,event_type
0,118,2,2,312,19,34
4,91,2,2,315,200,34
8,152,2,2,221,1,35
12,931,1,8,80,9,15
18,120,1,2,134,1,34


In [21]:
x=train4

### Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [23]:
g=GaussianNB()
b=BernoulliNB()

In [24]:
g.fit(x,y)
b.fit(x,y)

BernoulliNB()

In [25]:
tahmin1=g.predict(x)
tahmin2=b.predict(x)

#### Metrics

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [27]:
accuracy_score(y,tahmin1), precision_score(y,tahmin1,average=None), recall_score(y,tahmin1,average=None), f1_score(y,tahmin1,average=None)

(0.5210676060154451,
 array([0.78871201, 0.32185273, 0.22      ]),
 array([0.56960702, 0.43452699, 0.42424242]),
 array([0.66148804, 0.36979759, 0.289746  ]))

In [28]:
accuracy_score(y,tahmin2), precision_score(y,tahmin2,average=None), recall_score(y,tahmin2,average=None), f1_score(y,tahmin2,average=None)

(0.6481506570925348,
 array([0.64815066, 0.        , 0.        ]),
 array([1., 0., 0.]),
 array([0.7865187, 0.       , 0.       ]))

### K-Nearest Neighbors

In [29]:
from sklearn.neighbors import KNeighborsClassifier
k=KNeighborsClassifier()

In [30]:
k.fit(x,y)
tahmin3=k.predict(x)

#### Metrics

In [31]:
accuracy_score(y,tahmin3), precision_score(y,tahmin3,average=None), recall_score(y,tahmin3,average=None), f1_score(y,tahmin3,average=None)

(0.7564015716027639,
 array([0.78666182, 0.66065464, 0.69553806]),
 array([0.9048913 , 0.52859433, 0.36501377]),
 array([0.84164479, 0.58729216, 0.47877145]))

### Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression
l=LogisticRegression()

In [33]:
l.fit(x,y)
tahmin4=l.predict(x)

#### Metrics

In [34]:
accuracy_score(y,tahmin4), precision_score(y,tahmin4,average=None), recall_score(y,tahmin4,average=None), f1_score(y,tahmin4,average=None)

(0.6470667931174637,
 array([0.6484832 , 0.46153846, 0.17647059]),
 array([0.99644649, 0.00320684, 0.00413223]),
 array([0.78566131, 0.00636943, 0.00807537]))

### Gradient Boost Classifier (Boosting)

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()

In [36]:
gb.fit(x,y)
tahmin5=gb.predict(x)

#### Metrics

In [37]:
accuracy_score(y,tahmin5), precision_score(y,tahmin5,average=None), recall_score(y,tahmin5,average=None), f1_score(y,tahmin5,average=None)

(0.7391952309985097,
 array([0.75879828, 0.66987741, 0.65458937]),
 array([0.92391304, 0.40887226, 0.37327824]),
 array([0.83325478, 0.50779954, 0.4754386 ]))

### Random Forest Classifier (Bagging)

In [38]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

In [39]:
rf.fit(x,y)
tahmin6=rf.predict(x)

#### Metrics

In [40]:
accuracy_score(y,tahmin6), precision_score(y,tahmin6,average=None), recall_score(y,tahmin6,average=None), f1_score(y,tahmin6,average=None)

(0.9697872916948923,
 array([0.979375  , 0.95461826, 0.94491525]),
 array([0.9826505, 0.9556387, 0.9214876]),
 array([0.98101002, 0.95512821, 0.93305439]))

###  DecisionTreeClassifier

In [41]:
from sklearn.tree import DecisionTreeClassifier
dc=DecisionTreeClassifier()

In [42]:
dc.fit(x,y)
tahmin7=dc.predict(x)

#### Metrics

In [43]:
accuracy_score(y,tahmin7), precision_score(y,tahmin7,average=None), recall_score(y,tahmin7,average=None), f1_score(y,tahmin7,average=None)

(0.9697872916948923,
 array([0.96709992, 0.97125484, 0.98611111]),
 array([0.99540134, 0.93907002, 0.88016529]),
 array([0.98104656, 0.9548913 , 0.930131  ]))